In [2]:
# Loading all the needed Packages
import pandas as pd
import numpy as np

In [25]:
# Load all CSV files

Dem_data = pd.read_excel('../Data/Data_demand.xlsx', sheet_name='Python_dem')
Dem_U = pd.read_excel('../Data/Data_demand.xlsx', sheet_name='Python_U')

In [26]:
Dem_data


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,73.416544,65.688487,121.716902,50.232372,48.300358,92.736687,85.008630,115.920859,117.852873,131.376974,179.677332,131.376974,214.453589,67.620501,226.045675,123.648916,86.940644
1,68.102743,60.934033,112.907178,46.596613,44.804436,86.024517,78.855807,107.530646,109.322824,121.868066,166.672502,121.868066,198.931695,62.726210,209.684760,114.699356,80.647985
2,64.840894,58.015537,107.499377,44.364822,42.658483,81.904287,75.078930,102.380359,104.086698,116.031074,158.689557,116.031074,189.403664,59.721876,199.641700,109.205716,76.785269
3,64.474130,57.687380,106.891321,44.113879,42.417191,81.441007,74.654256,101.801258,103.497946,115.374760,157.791951,115.374760,188.332328,59.384067,198.512454,108.588009,76.350944
4,65.811549,58.884018,109.108621,45.028955,43.297072,83.130378,76.202846,103.912972,105.644855,117.768035,161.065107,117.768035,192.238999,60.615901,202.630296,110.840504,77.934729
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3595,156.000690,139.579565,258.632724,106.737315,102.632033,197.053504,180.632378,246.316880,250.422161,279.159130,381.791164,279.159130,455.686227,143.684846,480.317915,262.738005,184.737660
3596,155.112590,138.784949,257.160347,106.129667,102.047757,195.931693,179.604052,244.914616,248.996526,277.569898,379.617655,277.569898,453.092039,142.866859,477.583501,261.242257,183.685962
3597,140.885156,126.055139,233.572758,96.395107,92.687602,177.960197,163.130180,222.450246,226.157750,252.110279,344.797881,252.110279,411.532955,129.762643,433.777979,237.280262,166.837684
3598,120.169366,107.519959,199.228160,82.221145,79.058794,151.792884,139.143477,189.741105,192.903457,215.039919,294.098712,215.039919,351.021044,110.682311,369.995154,202.390512,142.305829


In [ ]:
Dem_data @ Dem_U

,U
0,7.592720e+05
1,7.043168e+05
2,6.705828e+05
3,6.667898e+05
4,6.806213e+05
...,...
3595,1.613355e+06
3596,1.604170e+06
3597,1.457031e+06
3598,1.242788e+06


In [ ]:
z_dest = model.addMVar(hours, lb=0) # Auxiliary variable

max_cap = model.addConstr( p_cap <= max_capacities, name='max_cap')

objective = p_cap @ cost_cap + e_buy @ Prices + m_CO2_buy @ CO2_prices

